# Multi-tenant Chat with Papers - Setup
## Get keys and urls

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY[:20]}")
print(f"OpenAI URL: {OPENAI_URL}")

Weaviate Key:root-user-key
OpenAI API Key: sk-proj-iuwKF1Q94jnW
OpenAI URL: https://api.openai.com


## Connect to Weaviate

In [2]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

True


## Create Tenant-ready collection

In [3]:
from weaviate.classes.config import Configure

if (client.collections.exists("Papers")):
    client.collections.delete("Papers")

client.collections.create(
    "Papers",

    vector_config=[
        Configure.Vectors.text2vec_openai(
            name="main_vector",

            model="text-embedding-3-small",
            base_url=OPENAI_URL,
            source_properties=["chunk"]
        )
    ],

    # supported models: https://weaviate.io/developers/weaviate/model-providers/openai/generative#available-models
    generative_config=Configure.Generative.openai(
        model="gpt-4o-mini",
        base_url=OPENAI_URL 
    ),

    multi_tenancy_config=Configure.multi_tenancy(True)
)

## List Tenants

In [4]:
papers = client.collections.get("Papers")
papers.tenants.get()

{}

## Close the client

In [5]:
client.close()